In [1]:
exec(open("../../../python/FNC_init.py").read())

[**Demo %s**](#demo-condition-bound)


```{index} ! Python; cond
```

The function `cond` from `numpy.linalg` is used to computes matrix condition numbers. By default, the 2-norm is used. As an example, the family of *Hilbert matrices* is famously badly conditioned. Here is the $6\times 6$  case.

In [2]:
A = array([ 
    [1/(i + j + 2) for j in range(6)] 
    for i in range(6) 
    ])
print(A)

[[0.5        0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.33333333 0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.25       0.2        0.16666667 0.14285714 0.125      0.11111111]
 [0.2        0.16666667 0.14285714 0.125      0.11111111 0.1       ]
 [0.16666667 0.14285714 0.125      0.11111111 0.1        0.09090909]
 [0.14285714 0.125      0.11111111 0.1        0.09090909 0.08333333]]


In [3]:
from numpy.linalg import cond
kappa = cond(A)
print(f"kappa is {kappa:.3e}")

kappa is 5.110e+07


Next we engineer a linear system problem to which we know the exact answer.

In [4]:
x_exact = 1.0 + arange(6)
b = A @ x_exact

Now we perturb the data randomly with a vector of norm $10^{-12}$.

In [5]:
dA = random.randn(6, 6)
dA = 1e-12 * (dA / norm(dA, 2))
db = random.randn(6)
db = 1e-12 * (db / norm(db, 2))

We solve the perturbed problem using built-in pivoted LU and see how the solution was changed.

In [6]:
x = linalg.solve(A + dA, b + db) 
dx = x - x_exact

Here is the relative error in the solution.

In [7]:
print(f"relative error is {norm(dx) / norm(x_exact):.2e}")

relative error is 5.01e-06


And here are upper bounds predicted using the condition number of the original matrix.

In [8]:
print(f"b_bound: {kappa * 1e-12 / norm(b):.2e}")
print(f"A_bound: {kappa * 1e-12 / norm(A, 2):.2e}")

b_bound: 6.72e-06
A_bound: 4.57e-05


Even if we don't make any manual perturbations to the data, machine epsilon does when we solve the linear system numerically.

In [9]:
x = linalg.solve(A, b)
print(f"relative error: {norm(x - x_exact) / norm(x_exact):.2e}")
print(f"rounding bound: {kappa / 2**52:.2e}")


relative error: 6.85e-10
rounding bound: 1.13e-08


Because $\kappa\approx 10^8$, it's possible to lose 8 digits of accuracy in the process of passing from $A$ and $b$ to $x$. That's independent of the algorithm; it's inevitable once the data are expressed in double precision. 

Larger Hilbert matrices are even more poorly conditioned.

In [10]:
A = array([ [1/(i+j+2) for j in range(14)] for i in range(14) ])
kappa = cond(A)
print(f"kappa is {kappa:.3e}")

kappa is 1.436e+19


Before we compute the solution, note that $\kappa$ exceeds `1/eps`. In principle we therefore might end up with an answer that is completely wrong (i.e., a relative error greater than 100%).

In [11]:
print(f"rounding bound: {kappa / 2**52:.2e}")

rounding bound: 3.19e+03


In [12]:
x_exact = 1.0 + arange(14)
b = A @ x_exact  
x = linalg.solve(A, b)

We got an answer. But in fact, the error does exceed 100%:

In [13]:
print(f"relative error: {norm(x - x_exact) / norm(x_exact):.2e}")

relative error: 1.42e+01
